In [1]:
import pickle
import sys

sys.path.append("../")
import numpy as np
import optuna
import pandas as pd
from numpy import random
from scipy.stats import qmc
from sklearn.model_selection import cross_val_score

sys.path.append('../src')

import matplotlib.pyplot as plt
import shap
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from utils import plot_pca
from xgboost import XGBRegressor
from sklearn.tree unoirt DecisionTreeRegressor

from straindesignxai.main import DataLoader

path = "../data/simulated/sim_data_FM_cycle1.csv"
data = DataLoader(path, 'y')
print(data)



Dataset size: (16, 11)


In [ ]:
data.get_shap_values()
data.supervised_clustering()

In [ ]:
# # Number of elements in each cluster
# print('NUMBER OF ELEMENTS IN EACH CLUSTER')
# display(data.df['cluster'].value_counts())
# # groupy by cluster and calculate most common values
# print('MOST COMMON VALUES')
# display(data.df.groupby('cluster').agg(lambda x:x.value_counts().index[0]))
# # groupy by cluster and calculate mean values
# print('MEAN VALUES')
# display(data.df.groupby('cluster').mean())
# # groupy by cluster and calculate std values
# print('STD VALUES')
# display(data.df.groupby('cluster').std())
# # groupy by cluster and calculate median values
# print('MEDIAN VALUES')
# display(data.df.groupby('cluster').median())

In [ ]:
# Get rows in cluster 3 and 5
df = data.df[data.df['cluster'].isin([4,5])]
df.describe()

In [ ]:
def true_function_medium(x, dimension):
    f = 0.0
    for i in range(dimension):
        f += x[i] ** 4 - 16 * x[i] ** 2 + 5 * x[i]

    f *= 1 / dimension
    return -1.0 * f


In [ ]:
N = 100
sim_df = pd.DataFrame(columns=data.X.columns)
for col in data.X.columns:
    sim_df[col] = np.random.normal(df.describe()[col]['mean'], df.describe()[col]['std'], N)    

sim_df['pred'] = data.model.predict(sim_df)
sim_df.sort_values(by='pred', ascending=False, inplace=True)    
sim_df.head()

In [ ]:
# x = data.X.iloc[2].values
x = sim_df.iloc[0].values
f=0.0
for i in range(10):
    f += x[i] ** 4 - 16 * x[i] ** 2 + 5 * x[i]
f *= 1 / 10
f = - 1 * f
f

In [ ]:
import pickle
import sys

import numpy as np
import optuna
import pandas as pd
from scipy.stats import qmc
from sklearn.model_selection import cross_val_score

sys.path.append('../src')

import matplotlib.pyplot as plt
import shap
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from utils import plot_pca
from xgboost import XGBRegressor


In [ ]:
# No. of input variables (features)
user_params = {"dim": 10}

input_vars = ["x_" + str(i) for i in range(1, user_params["dim"] + 1)]
response_vars = ["y"]

# Define no. of instances for the initial DBTL cycle
user_params["n_instances_cycle1"] = 100
# Define the no. of cycles to perform
user_params["num_recommendations"] = 16
# Define the objective for optimization and a threshold for defining success (for example, a success is 20% improvement over the best in the training data set)
user_params["objective"] = "maximize"
user_params["threshold"] = 0.2


### Define true response function

In [ ]:
def true_function_medium(x, dimension):
    f = 0.0
    for i in range(dimension):
        f += x[i] ** 4 - 16 * x[i] ** 2 + 5 * x[i]

    f *= 1 / dimension
    return -1.0 * f


In [ ]:
dim = user_params["dim"]
global_optimum_value = 78.332
global_optimum = -2.903534 * np.ones(dim)
lb, ub = -5.0, 5.0


#### Define constraints on the input variables

In [ ]:
bounds = pd.DataFrame(columns=["Variable", "Min", "Max", "Scaling"])
bounds["Variable"] = ["x_" + str(i) for i in range(1, dim + 1)]
bounds["Min"] = lb * np.ones(dim)
bounds["Max"] = ub * np.ones(dim)
bounds["Scaling"] = np.ones(dim)
bounds = bounds.set_index("Variable")


## Cycle 1

In [ ]:
cycle = 1

In [ ]:
def create_initial_cycle_data(bounds: pd.DataFrame, user_params: dict) -> pd.DataFrame:
    """ Generate data for initial design

    Args:
        bounds (pd.DataFrame)
        user_params (dict)

    Returns:
        pd.DataFrame
    """
    sampler = qmc.LatinHypercube(d=user_params["dim"])
    sample = sampler.random(n=user_params["n_instances_cycle1"])
    sample = qmc.scale(sample, bounds["Min"].values, bounds["Max"].values)

    X_initial = pd.DataFrame(sample, columns=input_vars)
    y_initial = true_function_medium(X_initial.values.T, dim).reshape(-1, 1)

    df = pd.DataFrame(
        np.hstack((X_initial, y_initial)), columns=input_vars + response_vars
    )
    return df

In [ ]:
# Read cycle 1 data from ART paper
df = pd.read_csv("../data/simulated/sim_data_FM_cycle1.csv")
cols = df["Type"].unique()
idx = df["Line Name"].unique()
data = pd.DataFrame(columns=cols, index=idx)
for i in idx:
    for j in cols:
        data.loc[i, j] = df[(df["Line Name"] == i) & (df["Type"] == j)]["Value"].values[0]

data = data.astype(float)

X = data.drop("y", axis=1)
y = data["y"]

print(data.shape)
data.head()

In [ ]:
# Train and tune an XGBoost model using optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda',1e-4, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'n_jobs': -1,
        'random_state': 42
    }
    
    xgb = XGBRegressor(**params)
    scores = cross_val_score(xgb, X, y, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
    return -scores.mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Train the model on the entire dataset
params = study.best_trial.params
xgb_model = XGBRegressor(**params)
xgb_model.fit(X, y)


In [ ]:
# Cross validate model on the data
def train(model, X, y):
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
    print(f'RMSE = {np.round(np.sqrt(np.abs(scores.mean())),4)}')
    print(f'STD = {np.round(scores.std(),4)}')
    model.fit(X, y)
    return model

def plot_R2(model, X, y):
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.scatter(y, model.predict(X), color='blue', alpha=0.5)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title('Cross Validation Predictions')
    plt.show()

xgb = train(xgb_model, X, y)
plot_R2(xgb, X, y)

In [ ]:
def shap_plots(model, X):
    # Calculate SHAP values and plot
    explainer = shap.TreeExplainer(model, X)
    shap_values = explainer(X)
    shap.summary_plot(shap_values, X, plot_type='bar')
    shap.summary_plot(shap_values, X, plot_type='dot')
    order = np.argsort(model.predict(X))
    shap.plots.heatmap(shap_values, instance_order=order)
    return explainer, shap_values

explainer, shap_values = shap_plots(xgb, X)
# Shap values to dataframe
shap_df = pd.DataFrame(shap_values.values, columns=X.columns)
shap_df.index = data.index

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(2,10))
visualizer.fit(shap_df)
visualizer.show()

# Get optimal number of clusters
n_clusters = visualizer.elbow_value_
print(f'Optimal number of clusters = {n_clusters}')

kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42).fit(shap_df)
shap_df['Cluster'] = kmeans.labels_
shap_df['y'] = data['y']

In [ ]:
data['Cluster'] = shap_df['Cluster']
# groupy by cluster and calculate most common values
print('MOST COMMON VALUES')
display(data.groupby('Cluster').agg(lambda x:x.value_counts().index[0]))
# groupy by cluster and calculate mean values
print('MEAN VALUES')
display(data.groupby('Cluster').mean())
# groupy by cluster and calculate std values
print('STD VALUES')
display(data.groupby('Cluster').std())
# groupy by cluster and calculate median values
print('MEDIAN VALUES')
display(data.groupby('Cluster').median())

In [ ]:
data[data['Cluster'] == 3].sort_values(by='y', ascending=False)